In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# Read in all metadata (check)
# Reduce the metadata to just the faulty clips (check)
# For each AM, subtract out their start time
# Multiply their times by four
# Add back in the start time
# Convert back to standard time UTC

In [3]:
metadata_df = pd.read_csv("MadreDeDios_Audiomoth_Dataset_Metadata.csv")

C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (0,1,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# This dictionary has its keys as the names of the faulty Audiomoths, and the values as the recorded turn-on time
## datetime.datetime(year,month,day,hour,minute,second)
# https://www.geeksforgeeks.org/python-difference-between-two-dates-in-minutes-using-datetime-timedelta-method/
## TODO
# Readjust these to use the "Programming Time" Column, rather than the "File" column
## DONE
bad_am_dict = {"AM-20" : datetime.datetime(2019,6,9,9,23,0),
               "AM-3" :  datetime.datetime(2019,6,9,23,8,0),
               "AM-4" :  datetime.datetime(2019,6,9,23,16,0),
               "AM-5" :  datetime.datetime(2019,6,10,0,10,0),
               "AM-6" :  datetime.datetime(2019,6,9,22,6,0),
               "AM-7" :  datetime.datetime(2019,6,9,7,13,0),
               "AM-8" :  datetime.datetime(2019,6,9,23,23,0),
               "AM-9" :  datetime.datetime(2019,6,9,23,11,0),
               "AM-10" : datetime.datetime(2019,6,10,0,5,0),
               "AM-11" : datetime.datetime(2019,6,10,0,45,0),
               "AM-12" : datetime.datetime(2019,6,10,0,43,0),
               "AM-13" : datetime.datetime(2019,6,10,0,48,0),
               "AM-14" : datetime.datetime(2019,6,10,0,36,0),
               "AM-15" : datetime.datetime(2019,6,10,1,3,0),
               "AM-16" : datetime.datetime(2019,6,10,0,50,0),
               "AM-17" : datetime.datetime(2019,6,10,1,8,0),
               "AM-18" : datetime.datetime(2019,6,9,23,25,0)
# Two other faulty recordings, that didn't have an initial timestamp that I have received  
# I could potentially just use the first time on the deployment date, TBD
#               "AM-23" : datetime.datetime(2019),
#               "WWF-2" : datetime.datetime()
              }

In [5]:
# Creating a list from the dictionary keys 
bad_am_list = list(bad_am_dict.keys())
# Filtering out everything that isn't one of the Audiomoths listed as bad from the metadata
metadata_df_reduced = metadata_df[metadata_df['AudioMothCode'].isin(bad_am_list)]
# Filtering out clips that failed to receive a Microfaune score (faulty clips)
metadata_df_reduced = metadata_df_reduced[metadata_df_reduced["Global_Score"] != -1]

In [6]:
metadata_df_reduced.reset_index(drop=True,inplace=True)

In [7]:
# Initializing a dictionary to contain the timing corrections
timing_correction_df = {
    "AudioMoth"            : [],
    "FileName"             : [],
    "IncorrectDate"        : [],
    "IncorrectElapsedTime" : [],
    "CorrectElapsedTime"   : [],
    "CorrectDate"          : []
}
corrected_df = pd.DataFrame()
# pulling out the date and time of recording according to the "Comment" Column
# Going through each device in the list
## TODO, rework so that the metadata is completely reworked to have the correct times all in one column.
# This will require you to parse the filename directly.
for am in bad_am_list:
    am_df = metadata_df_reduced[metadata_df_reduced["AudioMothCode"] == am]
    # Looping through each row in the Audiomoth dataframe
    for row in am_df.index:
        comment = am_df["Comment"][row]
        #print(comment)
        comment_split = comment.split(' ') 
        time = comment_split[2]
        date = comment_split[3]
        #print("Time : ",time, "  Date: ",date)
        time_split = time.split(':')
        date_split = date.split('/')
        # Breaking down the time of recording
        hour = int(time_split[0])
        minute = int(time_split[1])
        second = int(time_split[2])
        # Breaking down the data of recording
        day = int(date_split[0])
        month = int(date_split[1])
        #print(month)
        year = int(date_split[2])
        ## datetime.datetime(year,month,day,hour,minute,second)
        formatted_time = datetime.datetime(year,month,day,hour,minute,second)
        incorrect_elapsed_time = formatted_time-bad_am_dict[am]
        # The metadata says that they are recording 1 minute, every 10 minutes
        # In reality, they were recording 1 minute, every 40 minutes
        corrected_elapsed_time = incorrect_elapsed_time * 4 
        # Pretty Printing
        if am == "AM-3" and row % 200 == 0:
            print("\nAudioMoth Clip: ",am_df["FileName"][row])
            print("Incorrect Elapsed Time: ",incorrect_elapsed_time, "\nCorrected Elapsed Time",corrected_elapsed_time)
            print("Incorrect Data and Time", bad_am_dict[am]+incorrect_elapsed_time,"\nCorrected Date and Time: ", bad_am_dict[am]+corrected_elapsed_time)
        # Inserting into dictionary for long-term storage
        timing_correction_df["AudioMoth"].append(am_df["AudioMothCode"][row])
        timing_correction_df["FileName"].append(am_df["FileName"][row])
        timing_correction_df["IncorrectDate"].append(bad_am_dict[am]+incorrect_elapsed_time)
        timing_correction_df["IncorrectElapsedTime"].append(incorrect_elapsed_time)
        timing_correction_df["CorrectElapsedTime"].append(corrected_elapsed_time)
        timing_correction_df["CorrectDate"].append(bad_am_dict[am]+corrected_elapsed_time)
        # Modifying the dataframe directly
        am_df["IncorrectDate"][row] = bad_am_dict[am]+incorrect_elapsed_time
        am_df["IncorrectElapsedTime"][row] = incorrect_elapsed_time
        am_df["CorrectElapsedTime"][row] = corrected_elapsed_time
        am_df["CorrectDate"][row] = bad_am_dict[am]+corrected_elapsed_time
        if corrected_df.empty:
            corrected_df = am_df
        else:
            corrected_df = corrected_df.append(am_df)
        

C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\pandas\core\series.py:1031: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_labels(key, value)
C:\Users\17149\miniconda3\envs\species-id\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e


AudioMoth Clip:  20190611_215000.WAV
Incorrect Elapsed Time:  1 day, 22:42:00 
Corrected Elapsed Time 7 days, 18:48:00
Incorrect Data and Time 2019-06-11 21:50:00 
Corrected Date and Time:  2019-06-17 17:56:00

AudioMoth Clip:  20190613_071000.WAV
Incorrect Elapsed Time:  3 days, 8:02:00 
Corrected Elapsed Time 13 days, 8:08:00
Incorrect Data and Time 2019-06-13 07:10:00 
Corrected Date and Time:  2019-06-23 07:16:00

AudioMoth Clip:  20190614_163000.WAV
Incorrect Elapsed Time:  4 days, 17:22:00 
Corrected Elapsed Time 18 days, 21:28:00
Incorrect Data and Time 2019-06-14 16:30:00 
Corrected Date and Time:  2019-06-28 20:36:00

AudioMoth Clip:  20190616_015000.WAV
Incorrect Elapsed Time:  6 days, 2:42:00 
Corrected Elapsed Time 24 days, 10:48:00
Incorrect Data and Time 2019-06-16 01:50:00 
Corrected Date and Time:  2019-07-04 09:56:00

AudioMoth Clip:  20190617_111000.WAV
Incorrect Elapsed Time:  7 days, 12:02:00 
Corrected Elapsed Time 30 days, 0:08:00
Incorrect Data and Time 2019-06-

KeyboardInterrupt: 

In [ ]:
corrected_df.reset_index(drop=True,inplace=True)
corrected_df.to_csv("FaultyTimingAMs_Corrected.csv", index=False)